# Using Local Grid Refinement (LGR) to create a fully connected inset (child) model within a parent. 

In [ ]:
import sys, os
import numpy as np
import flopy as fp
from flopy.utils.lgrutil import Lgr
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

from matplotlib.backends.backend_pdf import PdfPages

## set up a directory to write the files in and run in.

In [ ]:
model_dir = 'theis_flopy_mf6_LGR'
model_name = 'theis'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [ ]:
# create simulation
sim = fp.mf6.MFSimulation(sim_name='theis_mf6', version='mf6', exe_name='mf6', 
                             sim_ws=model_dir)


In [ ]:
# create tdis package
tdis_rc = [(86400.0, 75, 1.2)]
tdis = fp.mf6.ModflowTdis(sim, pname='tdis', time_units='DAYS', 
                             perioddata=tdis_rc)

## create parent model - we will make this with coarse discretization

In [ ]:
# create gwf model
gwf = fp.mf6.ModflowGwf(sim, modelname=model_name,
                           model_nam_file='{}.nam'.format(model_name))
gwf.name_file.save_flows = True

In [ ]:
# create iterative model solution and register the gwf model with it
ims = fp.mf6.ModflowIms(sim, pname='ims', print_option='SUMMARY', 
                           complexity='SIMPLE', outer_hclose=1.e-2, 
                           outer_maximum=10, under_relaxation='NONE', 
                           inner_maximum=10, inner_hclose=1.e-4, 
                           rcloserecord=0.1, linear_acceleration='BICGSTAB', 
                           scaling_method='NONE', reordering_method='NONE', 
                           relaxation_factor=0.99)

In [ ]:
sim.register_ims_package(ims, [gwf.name])


In [ ]:
# dis
nlay = 1
nrow = 52
ncol = 52
delr = 1925
delc = 1925
top = 0
botm = -1

## We will use `idomain` to create our parent model with an inactive area (using number `0`, active areas are number `1`) where the child model will be placed

In [ ]:
#first create idomain with all value of 1
idomain = np.ones((nlay, nrow, ncol), dtype=np.int)
# then set the inset area to idomain = 0
idomain[0, 20:31, 20:31] = 0
# finally we can take a look at it
plt.matshow(idomain[0])

In [ ]:
# create the dis file using this defined idomain
dis = fp.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,idomain=idomain,
                              delr=delr, delc=delc,
                              top=top, botm=botm)

In [ ]:
# initial conditions
ic = fp.mf6.ModflowGwfic(gwf, pname='ic', strt=0.)

In [ ]:
# CHD equal to 0 at the boundaries
chd_spd = [[(0,0,i),0] for i in range(1,ncol)] + \
          [[(0,i,0),0] for i in range(nrow)] + \
          [[(0,nrow-1,i),0] for i in range(1,ncol)] + \
          [[(0,i,ncol-1),0] for i in range(1,nrow-1)]

In [ ]:
chd = fp.mf6.ModflowGwfchd(gwf, save_flows=True,stress_period_data = chd_spd)

In [ ]:
domain = np.zeros((nrow,ncol))

In [ ]:
for i in chd_spd:
    domain[i[0][1],i[0][2]] = 1

In [ ]:
plt.matshow(domain)
plt.colorbar()

In [ ]:
# npf 
hk=0.3

# node property flow
npf = fp.mf6.ModflowGwfnpf(gwf, 
                              icelltype=[0], 
                              k=[hk])

In [ ]:
# storage
ss=0.0008
stor = fp.mf6.ModflowGwfsto(gwf, ss=ss, transient=True)

In [ ]:
oc = fp.mf6.ModflowGwfoc(gwf, pname='oc', budget_filerecord='{}.cbc'.format(model_name),
                            head_filerecord='{}.hds'.format(model_name),
                            headprintrecord=[('COLUMNS', 10, 'WIDTH', 15,
                                              'DIGITS', 6, 'GENERAL')],
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
                            printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])

## now we define an `Lgr` object that is referenced to the geometry of the parent model. 

### `ncpp` defines how many child cells should be generated for each parent cell

### `ncppl` defines how many child layers should be generated for each parent layer

### the `Lgr` object creates the child model in the area of the parent where `idomain` is zero (e.g. inactive)

In [ ]:
lgr=Lgr(nlayp=nlay, nrowp=nrow, ncolp=ncol,
        delrp=delr,delcp=delc,topp=0,botmp=-1,
        idomainp=idomain,ncpp=20,ncppl=1)

## now we can create the child model and associate it with the `sim` object we already made

In [ ]:
# create child gwf model
cmodel_name = 'child'
cgwf = fp.mf6.ModflowGwf(sim, modelname=cmodel_name,
                           model_nam_file='child.nam')
cgwf.name_file.save_flows = True

In [ ]:
# the Lgr object provides many of the variables we need to define the child model
cnlay, cnrow, cncol = lgr.get_shape()
cdelr, cdelc = lgr.get_delr_delc()
ctop, cbotm = lgr.get_top_botm()
cidomain = lgr.get_idomain()

In [ ]:
# unlike a general model, we need to be explicit 
# about setting the origin coordinates for the child model to be sure it lines up with the parent properly
xorigin, yorigin = lgr.get_lower_left()
cgwf.modelgrid.set_coord_info(xoff=xorigin, yoff=yorigin)

In [ ]:
cdis = fp.mf6.ModflowGwfdis(cgwf, nlay=cnlay, nrow=cnrow, ncol=cncol,
                               delr=cdelr, delc=cdelc,
                               top=ctop, botm=cbotm, idomain=cidomain,
                               xorigin=xorigin, yorigin=yorigin)
cic = fp.mf6.ModflowGwfic(cgwf, pname='ic', strt=0.)
cnpf = fp.mf6.ModflowGwfnpf(cgwf,  
                              icelltype=[0], 
                              k=[hk])

# put the well in the center of the child model
welspd = [[(cnlay - 1, int(cnrow / 2), int(cncol / 2)), -1.16]]
cwel = fp.mf6.ModflowGwfwel(cgwf, print_input=True,
                              stress_period_data=welspd)

coc = fp.mf6.ModflowGwfoc(cgwf, pname='oc', budget_filerecord='{}.cbc'.format(cmodel_name),
                            head_filerecord='{}.hds'.format(cmodel_name),
                            headprintrecord=[('COLUMNS', 10, 'WIDTH', 15,
                                              'DIGITS', 6, 'GENERAL')],
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
                            printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])
stor = fp.mf6.ModflowGwfsto(cgwf, ss=ss, transient=True)

In [ ]:
cdis

## set up some observations

### note that the syntax of the filenames as dictionary keys is not documented very clearly


In [ ]:
cobs = fp.mf6.ModflowUtlobs(cgwf, digits=25,
                            continuous={'dd1.csv':[['obs1','drawdown',(0,111,111)],
                                                  ['obs2','drawdown',(0,211,211)],
                                                  ['obs3','drawdown',(0,113,113)]]}, pname='head_near_well',
                            filename='head_near_well.obs')

## Now that we've created the child model, we need to define the exchange between the parent and child. Two keywords (used as `auxiliary` variables ) need to be defined.

### `angldegx` defines the angle (in degrees) between the exchage face and the x-axis

### `cdist` defines the straight-line distance between cell centers of the parent and child. This is required when specific discharge is calculated (as it will be here)

### the `get_exchange` method of the `Lgr` object provides all of this in a way that can be easily passed into exchange object

In [ ]:
# exchange information
exchangedata = lgr.get_exchange_data(angldegx=True, cdist=True)
nexg = len(exchangedata)
gwfe = fp.mf6.ModflowGwfgwf(sim, exgtype='gwf6-gwf6', 
                               exgmnamea='theis', exgmnameb='child', 
                               nexg=nexg, auxiliary=[('angldegx', 'cdist')], 
                               exchangedata=exchangedata)

In [ ]:
exchangedata[0]

In [ ]:
# now write out the files and run the model
sim.write_simulation()
sim.run_simulation()

## We can visualize the parent model and see that the inset part is just inactive

In [ ]:
fname = os.path.join(model_dir, 'theis.hds')
hdobj = fp.utils.HeadFile(fname)
head = hdobj.get_alldata()

In [ ]:
plt.matshow(head[74,0,:,:])
plt.colorbar(shrink=0.8)

In [ ]:
ch = head[74,0,:,:].copy()
ch[ch>1e10]=np.nan
plt.matshow(ch)
plt.colorbar(shrink=0.8)

## Now that we have the child model as an independent model, we can ignore the parent for visualization purposes and just consider the child.

In [ ]:
fname = os.path.join(model_dir, 'child.hds')
hdobj = fp.utils.HeadFile(fname)
head = hdobj.get_alldata()

In [ ]:
head.shape

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mm = fp.plot.PlotMapView(cgwf, ax=ax)
v=mm.plot_array(head[74,0,:,:])
plt.xlim((48500,50000))
plt.ylim((50500,52000))
plt.colorbar(v)


In [ ]:
fig,ax = plt.subplots(figsize=(8,8))
mm = fp.plot.PlotMapView(cgwf, ax=ax)
#mm.plot_grid()
mm.plot_array(head[74,0,:,:], vmin=-2, vmax=0)
mmpar = fp.plot.PlotMapView(gwf, ax=ax)
h = mmpar.plot_array(ch, vmin=-2, vmax=0)
plt.colorbar(h)
plt.axis('equal')

In [ ]:
print_all = False
if print_all:
    with PdfPages('theis_mf_lgr.pdf') as pdfout:
        for i, chead in enumerate(head):
            printhead = np.squeeze(chead)
            plt.figure(figsize=(8,8))
            plt.imshow(#-printhead[100:120,100:120],
                       -printhead,
                      interpolation='nearest',
                      vmin=0,
                      vmax=2.7)
            plt.colorbar()
            plt.title('Theis-o-matic --> timestep{0}'.format(i))
            pdfout.savefig()
            plt.close()


## read in the observed drawdown at the points requested

In [ ]:
hd_ob = pd.read_csv('theis_flopy_mf6_LGR/dd1.csv', index_col=0)

## kludge in the case that some values are incorrectly formatted

In [ ]:
hd_ob['OBS2'].values

In [ ]:
for cc in hd_ob.columns:
    obs_cc = []
    for i in hd_ob[cc].values:
        try:
            obs_cc.append(float(i))
        except:
            obs_cc.append(np.nan)
    hd_ob[cc] = obs_cc

In [ ]:
hd_ob['OBS2'].values

In [ ]:
hd_ob.plot()